<a href="https://colab.research.google.com/github/Gooogr/Brain2Image/blob/master/sub_notebooks/BERT_fine_tuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial link: https://mccormickml.com/2019/07/22/BERT-fine-tuning/#52-evaluate-on-test-set <br>
Original clab link: https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX

In [ ]:
import os
import pandas as pd
import torch

In [ ]:
# ! nvidia-smi

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available.')
    print('Device name:', torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
    print("CUDA is not supported, using CPU.")

CUDA is available.
Device name: Tesla T4


### Prepare BERT from Hugging Face and CoLA dataset


In [ ]:
! pip install -q transformers

In [ ]:
cola_url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'


In [ ]:
# Download dataset if we don't have it
if not os.path.exists('./cola_public_1.1.zip'):
    ! wget $cola_url
    ! unzip -q ./cola_public_1.1.zip

--2020-09-22 14:41:27--  https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
Resolving nyu-mll.github.io (nyu-mll.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nyu-mll.github.io (nyu-mll.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255330 (249K) [application/zip]
Saving to: ‘cola_public_1.1.zip’

cola_public_1.1.zip 100%[===================>] 249.35K  --.-KB/s    in 0.03s   

2020-09-22 14:41:27 (8.52 MB/s) - ‘cola_public_1.1.zip’ saved [255330/255330]



In [ ]:
# Read raw train data. We need a raw data because of specific BERT tokenizer.
df = pd.read_csv('/content/cola_public/raw/in_domain_train.tsv',
                 delimiter='\t', 
                 header=None, 
                 names=['sentence_source', 'label', 'label_notes', 'sentence'])
print('Number of training sentences:', len(df))
df.sample(10)


Number of training sentences: 8551


,sentence_source,label,label_notes,sentence
8214,ad03,0,*,The owl hated the evil and the wise eagle.
2808,l-93,1,NaN,Paula swatted the fly with the cloth.
1493,r-67,0,*,I'll give to my good friend from Akron some.
6817,m_02,0,*,George built the houses both.
6895,m_02,1,NaN,Eleanor and Marianne travelled from Shropshire.
7391,sks13,1,NaN,I saw the student of physics with long hair.
367,bc01,1,NaN,Which picture of himself does Mary think that ...
4673,ks08,1,NaN,Ben can be relied on.
7492,sks13,1,NaN,Mary sent a book to Bill.
5049,ks08,1,NaN,That you came early surprised me.


In [ ]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values